# 4.7 分布式数据处理

分布式系统通常用于收集、访问和处理大型数据集。例如，本章前面介绍的数据库系统可以对存储在多台机器上的数据集进行操作。任何一台机器都可能不包含响应查询所需的数据，因此需要进行通信来处理请求。

本节将探讨一种典型的大数据处理场景，即数据集过大，无法由单台机器处理，而是分布在多台机器上，每台机器处理数据集的一部分。处理结果通常必须跨机器汇总，以便将一台机器的计算结果与其他机器的计算结果结合起来。为了协调这种分布式数据处理，我们将讨论一种名为MapReduce的编程框架。

使用 MapReduce 创建分布式数据处理应用程序结合了本文中介绍的许多思想。应用程序用纯函数来表示，这些函数用于映射（map）大型数据集，然后将映射的值序列还原（reduce）成最终结果。

函数式编程中的熟悉概念在 MapReduce 程序中得到了最大程度的应用。MapReduce 要求用于映射和还原数据的函数必须是纯函数。一般来说，仅用纯函数表示的程序在执行方式上具有相当大的灵活性。子表达式可以按照任意顺序并行计算，而不会影响最终结果。MapReduce 应用程序会并行评估许多纯函数，重新安排计算顺序，以便在分布式系统中高效执行。

MapReduce 的主要优势在于，它能在分布式数据处理应用程序的两个部分之间实现关注点分离：
- 1.处理数据和合并结果的映射和还原函数
- 2.机器之间的通信和协调

协调机制可以处理分布式计算中出现的许多问题，如机器故障、网络故障和进度监控。虽然管理这些问题会给 MapReduce 应用程序带来一些复杂性，但这些复杂性都不会暴露给应用程序开发人员。相反，构建 MapReduce 应用程序只需指定上述（1）中的映射和还原函数即可；分布式计算的挑战通过抽象被隐藏起来。

## 4.7.1 MapReduce

MapReduce 框架假定输入是任意类型的大量无序输入值流。例如，每个输入可能是某个庞大语料库中的一行文本。计算分三步进行。

1.对每个输入应用映射函数，输出零个或多个任意类型的中间键值对

3.所有中间键值对都按键分组，因此键值相同的键值对可以被还原在一起。

3.还原函数合并给定键 k 的值；它输出零个或多个值，每个值在最终输出中都与 k 相关联。

为了执行这一计算，MapReduce 框架创建了在计算中扮演不同角色的任务（可能在不同的机器上）。映射任务（map task）将映射函数应用于输入数据的某些子集，并输出中间键值对。还原（reduce）任务按键对键值进行排序和分组，然后对每个键的值应用还原函数。映射任务和还原任务之间的所有通信都由框架处理，按键对中间键值对进行分组的任务也是如此。

为了在 MapReduce 应用程序中利用多台机器，多个映射器在映射阶段（map phase）并行运行，多个还原器在还原阶段（reduce phase）并行运行。在这两个阶段之间，排序阶段（sort phase）通过排序将键值对组合在一起，从而使所有具有相同键值的键值对都相邻。

考虑一下计算文本语料库中元音的问题。我们可以使用 MapReduce 框架并适当选择 map 和 reduce 函数来解决这个问题。map 函数将一行文本作为输入，并输出键值对，其中键是 vowel，值是 count。输出中省略了零计数：


In [ ]:
def count_vowels(line):
    """A map function that counts the vowels in a line."""
    for vowel in 'aeiou':
        count = line.count(vowel)
        if count > 0:
            emit(vowel,count)

reduce 函数是 Python 内置的求和函数，它的输入是值的迭代器（给定键的所有值），并返回它们的和。